# The Partitura Package

partitura is python 3 package for symbolic music processing developed and maintained at CP/JKU (and otther contributors). It's intended to give a lightweight music representations that makes many score properties easily accessible for specific tasks. Furthermore it's a very useful I/O utility to parse computer formats of symbolic music such as MusicXML / MIDI. 

The repo can be found here (stable = main branch, bleeding edge = develop): https://github.com/CPJKU/partitura

Documentation can be found here: https://partitura.readthedocs.io/en/latest/

Extended tutorial notebooks are here: https://cpjku.github.io/partitura_tutorial/

# The part object

The ```part``` object is the central object of partitura. It contains a score.
- it is a timeline object
- time is measured in divs
- its elements are timed objects, i.e. they have a starting time and an ending time
- external score files are loaded into a part
- parts can be exported into score files
- it contains many useful methods related to its properties


In [ ]:
# Uncomment these lines if the kernel keeps crashing
# See https://stackoverflow.com/a/53014308
# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

import partitura as pt
path_to_musicxml = pt.EXAMPLE_MUSICXML
score = pt.load_musicxml(path_to_musicxml)
part = score[0]
print(part.pretty())
# what do we need to modify to see a somewhat pretty part?
# what do we see here?

![Timeline_chopin2.png](https://github.com/CPJKU/partitura_tutorial/raw/main/static/Timeline_chopin2.png)

## Notes

In [ ]:
part.notes

In [ ]:
# notes are timed objects
part.notes[0].start
# dir(part.notes[0])

In [4]:
# adding notes (times in divs!)
a_new_note = pt.score.Note(id='n04', step='A', octave=4, voice=1)
part.add(a_new_note, start=0, end=12)
# print(part.pretty())

In [5]:
# deleting notes
part.remove(a_new_note)
# print(part.pretty())

# how do you go about notes for which you don't have the corresponding object variable?

## Iterating over arbitrary musical objects

In [ ]:
for measure in part.iter_all(pt.score.Measure):
    print(measure)

In [ ]:
for note in part.iter_all(pt.score.Note, include_subclasses=True, start=0, end=24):
    print(note)

## The note_array

In [ ]:
# getting structured numpy array of notes from a part 
part.note_array()

In [ ]:
part.note_array()[["onset_beat", "pitch"]]

## File Import

In [10]:
# score representation format musicXML -> lets look at a file
score = pt.load_musicxml(path_to_musicxml)

In [11]:
path_to_midifile = pt.EXAMPLE_MIDI
performance = pt.load_performance_midi(path_to_midifile)

### excursion: what's a performed part?

In [ ]:
performance[0].notes

### excursion of the excursion: why isn't this a score representation?
musical notes representation format MIDI -> lets look at a file

In [ ]:
with open("example_data\\Chopin_op10_no3_p01.mid", 'rb') as f:
    # print(dir(f))
    for line in f:        
        print((line.hex()))
        print("---")

### part + performedpart + note alignment = match file

In [14]:
performance_m, alignment, score_m = pt.load_match("example_data\\Chopin_op10_no3_p01.match", create_score=True)

In [ ]:
alignment

## File export

In [16]:
pt.save_musicxml(part, "musicxml_out.xml")

In [17]:
pt.save_performance_midi(performance[0], "midi_out.mid")

### Some more imports for further processing

In [23]:
from partitura.musicanalysis.performance_codec import (encode_performance,
                                                        get_unique_onset_idxs,
                                                        notewise_to_onsetwise)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## A slightly more involved usage example

In [24]:
def match2tempo(match_path):
    """
    computes tempo curve from a match file.

    Parameters
    ----------
    match_path : string
        a path of a match file
    
    Returns
    -------
    tempo_curve_and_onsets : numpy array
        score onsets and corresponding tempo curve stacked as columns
    """

    perf, alignment, scor = pt.load_match(match_path, create_score=True)
    ppart, part = perf[0], scor[0]
    ppart.sustain_pedal_threshold = 128
    
    targets, snote_ids, ux = encode_performance(part, ppart, alignment, return_u_onset_idx=True)  
  
    nid_dict = dict((n.id, i) for i, n in enumerate(part.notes_tied))
    matched_subset_idxs = np.array([nid_dict[nid] for nid in snote_ids])       

    score_onsets = part.note_array()[matched_subset_idxs]['onset_beat'] # changed from onset_beat
    unique_onset_idxs, uni = get_unique_onset_idxs(score_onsets, return_unique_onsets=True)

    tmp_crv = notewise_to_onsetwise(np.array([targets["beat_period"]]).T, unique_onset_idxs)  
    tempo_curve_and_onsets = np.column_stack((uni, tmp_crv))

    return tempo_curve_and_onsets

In [25]:
te = match2tempo("example_data\\Chopin_op10_no3_p01.match")

In [ ]:
plt.plot(te[:,0], 60/te[:,1])